In [3]:
import os
import sys
import re
import pandas
import numpy as np
from functools import reduce
from collections import Counter, OrderedDict
from Bio import SeqIO

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


# Simple statistics for GFF and fasta files 

In [2]:
# Read GFF files, attach proper fasta sequence
scaffolds = dict.fromkeys([i for i in os.listdir(os.getcwd()) if i.endswith('.gff')])
for k in scaffolds.keys():
    scaffolds[k] = SeqIO.parse(k.replace('.gff', '.fasta'), 'fasta')
    
full_seqs = {}
for k,v in scaffolds.items():
    full_seqs[k] = reduce(lambda x, y: x + str(y.seq), v, "")
    full_seqs[k] = full_seqs[k].upper()

# Define couter
counts = {}
for k,v in full_seqs.items():
    counts[k] = dict(Counter([i for i in v]))

In [ ]:
# Count nucleotides in scaffolds
for k,v in counts.items():
    print(k, v)

In [ ]:
# Genome size (KB)
for k,v in full_seqs.items():
    print(k, len(v)/1000000)

In [ ]:
# Count genes
for k,v in genes.items():
    print(k, len(v))

In [ ]:
# Read GFF as dataframe, add headers
gffs = {}
for i in scaffolds.keys():
    print(i)
    gffs[i] = pandas.read_csv(i, sep='\t', header=None)

for k,v in gffs.items():
    gffs[k].columns = ['scaffold', '1', 'feature', 'start', 'end', '3', 'strand', '4', 'description']

# Hash all genes from distinct GFF
genes = {}
for k,v in gffs.items():
    genes[k] = gffs[k][(gffs[k].feature == 'gene')]

# Hash all mRNAs from distinct GFF
mrnas = {}
for k,v in gffs.items():
    mrnas[k] = gffs[k][(gffs[k].feature == 'mRNA')]


In [ ]:
# Calculate average exon length
exons = {}
for k,v in gffs.items():
    exons[k] = gffs[k][(gffs[k].feature == 'exon')]
    
exon_lens = {}
for k,v in exons.items():
    exon_lens[k] = v['end'].values - v['start'].values
for k,v in exon_lens.items():
    print(k, sum(v)/len(v))

# Exons per gene
for k in genes.keys():
    print(k, len(exons[k])/len(genes[k]))

## CDS features

In [ ]:
cdses = {}
for k,v in gffs.items():
    cdses[k] = gffs[k][(gffs[k].feature == 'CDS')]

for k,v in gffs.items():
    print(k)
    exon_starts, exon_ends = exons[k]['start'].values, exons[k]['end'].values
    cds_starts, cds_ends = cdses[k]['start'].values, cdses[k]['end'].values
    exon_positions = list(zip(exon_starts, exon_ends))
    cds_positions = list(zip(cds_starts, cds_ends))
    cds_positions.sort()
    exon_positions.sort()
    exon_positions = set(exon_positions)
    cds_positions = set(cds_positions)


exon_starts, exon_ends = exons[k]['start'].values, exons[k]['end'].values
cds_starts, cds_ends = cdses[k]['start'].values, cdses[k]['end'].values
exon_positions = list(zip(exon_starts, exon_ends))
cds_positions = list(zip(cds_starts, cds_ends))
cds_positions.sort()
exon_positions.sort()
exon_positions = set(exon_positions)
cds_positions = set(cds_positions)

exon_full_seqs = {}
for k,v in gffs.items():
    exon_full_seqs[k] = ''
    positions = zip(exons[k]['start'], exons[k]['end'])
    for e in positions:
        exon_full_seqs[k] += full_seqs[k][int(e[0])-1:int(e[1])]

In [ ]:
# extract intron positions from CDS
cds_positions = {}
for k,v in gffs.items():
    current_cds_positions = {}
    for i in gffs[k].iterrows():
        feature = i[1]
        if feature[2] == 'gene':
            current_gene = (int(feature[3]), int(feature[4]))
            current_cds_positions[current_gene] = []
        elif feature[2] == 'exon':
            current_cds_positions[current_gene].append((int(feature[3]), int(feature[4])))
    cds_positions[k] = current_cds_positions

intron_positions = {}
genes_with_introns = {}
for k,v in cds_positions.items():
    intron_positions[k] = []
    genes_with_introns[k] = 0
    for gene, cdses in v.items():
        current_intron_positions = []
        last_position = gene[0]
        for i in sorted(cdses):
            if i[0] > last_position:
                current_intron_positions.append((last_position, i[0]))
            last_position = i[1]
        if gene[1] > last_position and last_position != gene[0]:
            current_intron_positions.append((last_position, gene[1]))
        if len(current_intron_positions) > 0:
            genes_with_introns[k] += 1
        intron_positions[k].extend(current_intron_positions)

In [ ]:
# Intron length
intron_lens = {}
for k,v in intron_positions.items():
    intron_lens[k] = []
    for i in v:
        intron_lens[k].append(i[1]-i[0])

for k,v in intron_lens.items():
    print(k, sum(v)/len(v))
    
# Introns per gene
for k in genes.keys():
    print(k, len(intron[k])/len(genes[k]))   

# Define genomic features 

In [140]:
scaffold_seqs = {}
for k in scaffolds.keys():
    scaffolds[k] = SeqIO.parse(k.replace('.gff', '.fasta'), 'fasta')
for k, v in scaffolds.items():
    scaffold_seqs[k] = dict([(i.id, str(i.seq).upper()) for i in v])

In [161]:
exon_positions = {}
gene_positions = {}
for k in scaffolds.keys():
    exon_positions[k] = exons[k][['scaffold', 'start', 'end']]
    gene_positions[k] = genes[k][['scaffold', 'start', 'end']]

In [206]:
atcg_counts = {}
atcg_counts_introns = {}
atcg_counts_exons = {}
atcg_counts_intergenic = {}
for k, v in scaffold_seqs.items():
    atcg_counts[k] = {'A': 0, 'T': 0, 'G': 0, 'C': 0}
    atcg_counts_introns[k] = {'A': 0, 'T': 0, 'G': 0, 'C': 0}
    atcg_counts_exons[k] = {'A': 0, 'T': 0, 'G': 0, 'C': 0}
    atcg_counts_intergenic[k] = {'A': 0, 'T': 0, 'G': 0, 'C': 0}

In [ ]:
for k, v in scaffold_seqs.items():
    print(k)
    if k in ('Protothecka wickerhamii.gff', 'Helicosporidium sp.gff'):
        continue
    for scaffold_id, seq in v.items():
        print(scaffold_id)
        for i in enumerate(seq):
            if i[1] not in ('A', 'T', 'C', 'G'):
                continue
            if sum((exon_positions[k]['scaffold'] == scaffold_id) & (exon_positions[k]['start'] <= i[0]+1) & (exon_positions[k]['end'] >= i[0]+1)):
                atcg_counts_exons[k][i[1]] += 1
            elif sum((gene_positions[k]['scaffold'] == scaffold_id) & (gene_positions[k]['start'] <= i[0]+1) & (gene_positions[k]['end'] >= i[0]+1)):
                atcg_counts_introns[k][i[1]] += 1
            else:
                atcg_counts_intergenic[k][i[1]] += 1
            atcg_counts[k][i[1]] += 1

# Exon features

In [282]:
#Exon counts
atcg_counts_exon = {}
for k, v in scaffold_seqs.items():
    full_seq_feat = ''
    for exon in exon_positions[k].iterrows():
        full_seq_feat += scaffold_seqs[k][exon[1]['scaffold']][int(exon[1]['start'])-1:int(exon[1]['end'])]
    atcg_counts_exon[k] = dict(Counter(full_seq_feat))

# Intergenomic features

In [242]:
# Intergenic regions
atcg_counts_intergenic = {}
for k in scaffold_seqs.keys():
    atcg_counts_intergenic[k] = {}
    for i in 'ATCG':
        atcg_counts_intergenic[k][i] = atcg_counts[k][i] - atcg_counts_gene[k][i]

# Intron features

In [244]:
# Intron counts
atcg_counts_intron = {}
for k in scaffold_seqs.keys():
    atcg_counts_intron[k] = {}
    for i in 'ATCG':
        atcg_counts_intron[k][i] = atcg_counts_gene[k][i] - atcg_counts_exon[k][i]

# GC content

In [ ]:
# %GC content total
for k, v in atcg_counts.items():
    print(k, (v['G'] + v['C']) / sum([v['G'], v['C'], v['A'], v['T']]) * 100)

In [ ]:
# %GC content in exons
for k, v in atcg_counts_exon.items():
    print(k, (v['G'] + v['C']) / sum([v['G'], v['C'], v['A'], v['T']]) * 100)

In [ ]:
# %GC content in introns
for k, v in atcg_counts_intron.items():
    print(k, (v['G'] + v['C']) / sum([v['G'], v['C'], v['A'], v['T']]) * 100)

In [ ]:
# %GC content in intergenic regions
for k, v in atcg_counts_intergenic.items():
    print(k, (v['G'] + v['C']) / sum([v['G'], v['C'], v['A'], v['T']]) * 100)